# Check File place

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/aimnet2-x2d/pytorch/default/1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aimnet2-x2d/pytorch/default/1/multi_5props_best.pth


In [2]:
# List the input directory to find your dataset name
import os

# 1. Create the writable directories explicitly
os.makedirs("/kaggle/working/results", exist_ok=True)
print(os.listdir('/kaggle/input'))


['neurips-open-polymer-prediction-2025', 'mamba-whl', 'mamba-model', 'testset', 'kaggle-submission', 'aimnet-x2d-code', 'aimnet2-x2d']


# Eviroment Setup: Aimnet-X2d Script 

In [3]:
# 2. Set the PYTHONPATH so the script can find its own internal modules
# Replace 'aimnet-x2d-code' with the actual name of your uploaded dataset
env = os.environ.copy()
env["PYTHONPATH"] = "/kaggle/input/aimnet-x2d-code/AIMNet-X2D"

# Eviroment Setup: Aimnet-X2d Dependency

In [4]:
#  We can't directly run the main script through cli since we made some change on the dependancy.

In [5]:
import sys
import subprocess
import os
import glob
import shutil

# ==========================================
# CONFIGURATION
# ==========================================
INPUT_PATH = '/kaggle/input/kaggle-submission'
# This is where we want the library to live permanently for the launcher
FINAL_LIB_PATH = '/kaggle/working/external_libs'
os.makedirs(FINAL_LIB_PATH, exist_ok=True)

# ==========================================
# 1. HELPER: FIND SOURCE FOLDERS
# ==========================================
def find_package_path(base_name):
    search_pattern = os.path.join(INPUT_PATH, f"{base_name}*")
    matches = glob.glob(search_pattern)
    matches = [m for m in matches if os.path.basename(m) != "kaggle-submission"]
    return matches[0] if matches else None

# ==========================================
# 2. INSTALL DEPENDENCIES (RDKit + PyTorch Extras)
# ==========================================
def install_dependencies():
    print("🚀 Starting Dependency Installation...")
    
    # A. RDKit (Binary Wheel)
    rdkit_path = find_package_path("rdkit")
    if rdkit_path:
        try:
            import rdkit
            print("✅ RDKit is already installed.")
        except ImportError:
            print("Installing RDKit...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", rdkit_path, "--no-deps", "--no-index"])
    
    # B. PyTorch Scatter/Sparse/Cluster (Source)
    # These need to be installed via PIP so they compile
    pkgs = ["mhfp", "torch_scatter", "torch_sparse", "torch_cluster", "torch_spline_conv"]
    
    for pkg in pkgs:
        # Check if already installed to save time on re-runs
        try:
            __import__(pkg)
            print(f"✅ {pkg} is already installed.")
            continue
        except ImportError:
            pass

        print(f"Installing {pkg} (Compiling from source)...")
        pkg_path = find_package_path(pkg)
        if pkg_path:
            # We copy to a temp folder to build, avoiding read-only errors
            temp_build = f"/kaggle/working/build_{pkg}"
            if os.path.exists(temp_build): shutil.rmtree(temp_build)
            shutil.copytree(pkg_path, temp_build)
            
            # Find inner setup folder
            setup_dir = temp_build
            if not os.path.exists(os.path.join(temp_build, 'setup.py')):
                # Handle double nesting if present
                for child in os.listdir(temp_build):
                    if os.path.isdir(os.path.join(temp_build, child)):
                        setup_dir = os.path.join(temp_build, child)
                        break
            
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", 
                setup_dir, "--no-deps", "--no-index", "--no-build-isolation"
            ])

# ==========================================
# 3. SETUP TORCH GEOMETRIC (The "Move" Logic)
# ==========================================
def setup_torch_geometric():
    print("\n📦 Setting up PyTorch Geometric...")
    
    # If we already have it in the right place, skip
    target_path = os.path.join(FINAL_LIB_PATH, 'torch_geometric')
    if os.path.exists(target_path):
        print("✅ PyTorch Geometric is already set up in external_libs.")
        return

    # 1. Find the source
    pkg_path = find_package_path("torch_geometric")
    if not pkg_path:
        print("❌ Could not find torch_geometric source!")
        return

    # 2. Copy the source wrapper to a temp spot
    temp_extract = '/kaggle/working/temp_pyg_extract'
    if os.path.exists(temp_extract): shutil.rmtree(temp_extract)
    shutil.copytree(pkg_path, temp_extract)

    # 3. Locate the ACTUAL inner library folder 'torch_geometric'
    # It's usually inside: temp_extract/torch_geometric-2.7.0/torch_geometric
    inner_lib = None
    for root, dirs, files in os.walk(temp_extract):
        if 'torch_geometric' in dirs:
            candidate = os.path.join(root, 'torch_geometric')
            # Verify it contains __init__.py
            if os.path.exists(os.path.join(candidate, '__init__.py')):
                inner_lib = candidate
                break
    
    # 4. Move it to the final destination
    if inner_lib:
        shutil.copytree(inner_lib, target_path)
        print(f"✅ Moved torch_geometric to: {target_path}")
    else:
        print("❌ Failed to locate inner torch_geometric library folder.")

    # Cleanup temp
    shutil.rmtree(temp_extract)

# ==========================================
# EXECUTE
# ==========================================

install_dependencies()
setup_torch_geometric()
print("\n🎉 Environment Setup Complete!")

🚀 Starting Dependency Installation...
Installing RDKit...
Processing /kaggle/input/kaggle-submission/rdkit-2024.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Installing mhfp (Compiling from source)...
Processing ./build_mhfp/mhfp-1.9.6
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mhfp: filename=mhfp-1.9.6-py3-none-any.whl size=9735 sha256=a5c37dc44fa2001055c7d6a8c6f3101d61d402ba6f6106f9fa1a03b86c3993f7
  Stored in directory: /root/.cache/pip/wheels/84/69/20/4e5fa5a6a338c0afd28d36d71107eaa2adc0ecc0685a875ce2
Successfully built mhfp
Installing torch_scatter (Compiling from source)...
Processing ./build_torch_scatter/torch_scatter-2.1.2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3622728 sha256=10a26239c826f0cd426f1102f13d4008872331f038a0bbc76

# Dataset Manipulation (Compatible to AIMNET)

In [6]:
import pandas as pd
import os

# Read the test file
df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

# Rename column
if 'SMILES' in df.columns:
    df.rename(columns={'SMILES': 'smiles'}, inplace=True)

# Save to working directory for the model to read
os.makedirs('/kaggle/working/results', exist_ok=True)
df.to_csv('/kaggle/working/results/test.csv', index=False)
print("✅ Test file prepared at /kaggle/working/results/test.csv")

✅ Test file prepared at /kaggle/working/results/test.csv


# Real Script that WORK

In [7]:
%%writefile /kaggle/working/launcher.py
import sys
import os

# ==========================================
# 1. SETUP PATHS (The "Clean" Way)
# ==========================================
# We insert paths at index 0 to force Python to look here FIRST.
# This fixes the "datasets" collision without needing to delete modules.

# A. Path to AIMNet Source Code
aimnet_src = '/kaggle/input/aimnet-x2d-code/AIMNet-X2D/src'
sys.path.insert(0, aimnet_src)

# B. Path to PyTorch Geometric (External Libs)
ext_lib = '/kaggle/working/external_libs'
sys.path.insert(0, ext_lib)

# ==========================================
# 2. IMPORT MODULES
# ==========================================
try:
    # Now we can safely import. Python will find the local 'datasets' 
    # because we put 'src' at the top of the list.
    from main import main_runner, parse_main_arguments
    from config import validate_args
except ImportError as e:
    print(f"❌ Launcher Import Error: {e}")
    sys.exit(1)

# ==========================================
# 3. CONFIGURE ARGUMENTS
# ==========================================
# We explicitly set the arguments here
sys.argv = [
    'main.py',
    '--inference_csv', '/kaggle/working/results/test.csv',
    '--inference_output', '/kaggle/working/results/submission.csv',
    # Using the input path for loading weights is correct
    '--model_save_path', '/kaggle/input/aimnet2-x2d/pytorch/default/1/multi_5props_best.pth',
    # Force single-process data loading if possible to avoid pickle errors
]

# ==========================================
# 4. RUN INFERENCE
# ==========================================
if __name__ == "__main__":
    print("🚀 Starting Clean Inference Process...")
    try:
        args = parse_main_arguments()
        validate_args(args)
        main_runner(args)
        print("🎉 Launcher finished successfully!")
    except Exception as e:
        print(f"❌ Launcher Runtime Error: {e}")
        import traceback
        traceback.print_exc()

Writing /kaggle/working/launcher.py


In [8]:
!python /kaggle/working/launcher.py

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

# Post Prediction Compatibility Modification

In [9]:
# output_path = '/kaggle/working/results/submission.csv'

import pandas as pd
test_path = "/kaggle/working/results/test.csv"
output_path = "/kaggle/working/results/submission.csv"

df_with_id_smiles = pd.read_csv(test_path)
df_with_smiles_rowID_predictions = pd.read_csv(output_path)

df_aimnet = pd.merge(df_with_id_smiles, df_with_smiles_rowID_predictions, on='smiles', how='left')
df_aimnet = df_aimnet[[
    'id', 
    'Tg_prediction', 
    'FFV_prediction', 
    'Tc_prediction', 
    'Density_prediction', 
    'Rg_prediction'
]]

df_aimnet.rename(columns={'Tg_prediction': 'Tg',
                          'Tc_prediction':'Tc',
                          'Density_prediction':'Density',
                          'FFV_prediction':'FFV',
                          'Rg_prediction':'Rg'}, inplace=True)

df_aimnet.to_csv('/kaggle/working/submission.csv', index=False)
print(df_aimnet.head())
# df_with_id_smiles 

           id         Tg       FFV        Tc   Density         Rg
0  1109053969  73.775858  0.365856  0.209423  1.372850  19.089397
1  1422188626  76.749468  0.379604  0.239014  1.303095  19.183959
2  2032016830  48.316589  0.356741  0.280397  1.269335  18.412713


# XG Boost Training

In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import os


In [11]:

# ==========================================
# 1. CONFIGURATION
# ==========================================
TRAIN_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/train.csv' # Check this path!
TEST_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/test.csv'
AIMNET_SUB_PATH = '/kaggle/working/submission.csv' 
OUTPUT_PATH = '/kaggle/working/submission_ensemble.csv'

# Targets we need to predict
TARGETS = ['Tg', 'Tc', 'Density', 'FFV', 'Rg']

# ==========================================
# 2. HELPER: GENERATE FINGERPRINTS
# ==========================================
def get_fingerprints(smiles_list, radius=2, nBits=2048):
    fps = []
    for smi in smiles_list:
        try:
            mol = Chem.MolFromSmiles(smi)
            # Create bit vector
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            # Convert to numpy array
            arr = np.zeros((1,))
            AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
            fps.append(arr)
        except:
            # Fallback for bad SMILES (row of zeros)
            fps.append(np.zeros((nBits,)))
    return np.array(fps)

# ==========================================
# 3. LOAD DATA
# ==========================================
print("Loading data...")
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# Handle column renaming if needed
if 'SMILES' in train_df.columns: train_df.rename(columns={'SMILES': 'smiles'}, inplace=True)
if 'SMILES' in test_df.columns: test_df.rename(columns={'SMILES': 'smiles'}, inplace=True)

print(f"Generating Fingerprints for {len(train_df)} train and {len(test_df)} test molecules...")
# This might take 2-3 minutes
X_train = get_fingerprints(train_df['smiles'].values)
X_test = get_fingerprints(test_df['smiles'].values)

# ==========================================
# 4. TRAIN XGBOOST LOOP
# ==========================================
xgb_preds = np.zeros((len(test_df), len(TARGETS)))

print("Training XGBoost models...")
for i, target in enumerate(TARGETS):
    print(f"  Training for {target}...")
    
    # Get clean labels (drop NaNs for this specific target)
    mask = train_df[target].notna()
    y_train = train_df.loc[mask, target].values
    X_train_curr = X_train[mask]
    
    # Simple XGBoost Regressor
    model = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=4,
        tree_method='hist' # Fast mode
    )
    
    model.fit(X_train_curr, y_train)
    
    # Predict on Test
    xgb_preds[:, i] = model.predict(X_test)

# Create DataFrame for XGBoost predictions
df_xgb = pd.DataFrame(xgb_preds, columns=TARGETS)
df_xgb['id'] = test_df['id']

print(df_xgb.head())

Loading data...
Generating Fingerprints for 7973 train and 3 test molecules...


[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerator
[04:27:21] DEPRECATION WARNING: please use MorganGenerat

Training XGBoost models...
  Training for Tg...
  Training for Tc...
  Training for Density...
  Training for FFV...
  Training for Rg...
           Tg        Tc   Density       FFV         Rg          id
0  127.188515  0.236193  1.198138  0.369107  24.254484  1109053969
1  176.079651  0.243026  1.137916  0.375164  19.143675  1422188626
2   87.235199  0.282047  1.088306  0.353011  19.216194  2032016830


# Training a 1D Convolutional Neural Network (Treat Molecule as Language)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import re

# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ==========================================
# 1. IMPROVED TOKENIZER (Regex based)
# ==========================================
class SmilesTokenizer:
    def __init__(self):
        self.stoi = {}
        self.pad_token = "<pad>"
        self.unk_token = "<unk>"
        
    def regex_tokenize(self, smiles):
        pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        regex = re.compile(pattern)
        tokens = [token for token in regex.findall(smiles)]
        return tokens

    def fit(self, smiles_list):
        all_tokens = set()
        for s in smiles_list:
            all_tokens.update(self.regex_tokenize(s))
        
        # 0: Pad, 1: Unk
        vocab = [self.pad_token, self.unk_token] + sorted(list(all_tokens))
        self.stoi = {t: i for i, t in enumerate(vocab)}
        print(f"✅ Vocab Size: {len(self.stoi)}")
        
    def encode(self, smiles, max_len=128):
        tokens = self.regex_tokenize(smiles)
        vec = [self.stoi.get(t, self.stoi[self.unk_token]) for t in tokens]
        
        # Padding
        if len(vec) < max_len:
            vec += [self.stoi[self.pad_token]] * (max_len - len(vec))
        else:
            vec = vec[:max_len]
        return vec

# ==========================================
# 2. FIXED CNN MODEL (With Masking)
# ==========================================
class PolymerCNN(nn.Module):
    def __init__(self, vocab_size, output_dim=5, max_len=128):
        super().__init__()
        self.max_len = max_len
        self.embedding = nn.Embedding(vocab_size, 64, padding_idx=0)
        self.act = nn.LeakyReLU(0.1)
        self.conv1 = nn.Conv1d(64, 128, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=7, padding=3)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=7, padding=3)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, output_dim) # Linear Output (No Activation!)
        
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # x: [Batch, Seq_Len]
        # mask: [Batch, 1, Seq_Len] -> True where is NOT padding
        mask = (x != 0).unsqueeze(1).float().to(x.device)
        
        # Embedding & Permute
        h = self.embedding(x).permute(0, 2, 1) # [Batch, 64, Seq]
        
        # Conv Blocks with Masking applied *after* activation
        h = self.conv1(h)
        h = self.bn1(h)
        h = self.act(h)
        
        h = self.conv2(h)
        h = self.bn2(h)
        h = self.act(h)
        
        h = self.conv3(h)
        h = self.bn3(h)
        h = self.act(h)
        
        h = h.masked_fill(mask == 0, -1e9)
        
        # Global Max Pooling
        h = torch.max(h, dim=2)[0] # [Batch, 512]
        
        h = self.dropout(self.act(self.fc1(h)))
        out = self.fc2(h)
        
        return out

# ==========================================
# 3. TRAINING FUNCTION (Optimized)
# ==========================================
def train_cnn_model(train_df, test_df, targets, epochs=15, batch_size=64):
    print("🔹 Preparing Data for Fixed 1D CNN...")
    
    # Tokenize
    tokenizer = SmilesTokenizer()
    all_smiles = pd.concat([train_df['smiles'], test_df['smiles']]).astype(str).tolist()
    tokenizer.fit(all_smiles)
    
    X_train = torch.tensor([tokenizer.encode(s) for s in train_df['smiles']], dtype=torch.long)
    X_test = torch.tensor([tokenizer.encode(s) for s in test_df['smiles']], dtype=torch.long)
    
    # Scale Targets
    y_raw = train_df[targets].values
    scaler = StandardScaler()
    y_imputed = np.where(np.isnan(y_raw), np.nanmean(y_raw, axis=0), y_raw)
    y_train = torch.tensor(scaler.fit_transform(y_imputed), dtype=torch.float32)
    
    # Dataloader
    train_ds = torch.utils.data.TensorDataset(X_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    
    # Init Model
    model = PolymerCNN(len(tokenizer.stoi), output_dim=len(targets)).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)
    criterion = nn.MSELoss() 
    
    print(f"🔹 Training CNN for {epochs} epochs...")
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for xb, yb in train_dl:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}: Avg Loss {total_loss/len(train_dl):.4f}")
            
    return model, scaler, tokenizer, X_test

# ==========================================
# 4. INFERENCE
# ==========================================
def run_cnn_inference(model, scaler, X_test, test_ids, targets):
    print("🔹 Running Inference...")
    model.eval()
    test_ds = torch.utils.data.TensorDataset(X_test)
    test_dl = DataLoader(test_ds, batch_size=128)
    
    preds_list = []
    with torch.no_grad():
        for xb in test_dl:
            xb = xb[0].to(DEVICE)
            out = model(xb)
            preds_list.append(out.cpu().numpy())
            
    preds_scaled = np.vstack(preds_list)
    # 檢查預測是否全一樣 (Debug用)
    print("Debug Prediction Sample (Scaled):", preds_scaled[0])
    
    preds_orig = scaler.inverse_transform(preds_scaled)
    df_pred = pd.DataFrame(preds_orig, columns=targets)
    df_pred['id'] = test_ids
    return df_pred

# CNN train and inference

In [ ]:
import pandas as pd
# ==========================================
# 1. CONFIGURATION
# ==========================================
TRAIN_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/train.csv' # Check this path!
TEST_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/test.csv'

# Targets we need to predict
TARGETS = ['Tg', 'Tc', 'Density', 'FFV', 'Rg']

# ==========================================
# 2. LOAD DATA
# ==========================================
print("Loading data...")
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# Assuming you have loaded train_df and test_df already
# Ensure column name is 'smiles'
if 'SMILES' in train_df.columns: train_df.rename(columns={'SMILES': 'smiles'}, inplace=True)
if 'SMILES' in test_df.columns: test_df.rename(columns={'SMILES': 'smiles'}, inplace=True)

# # 1. Train the CNN
cnn_model, cnn_scaler, cnn_tokenizer, X_test_tensor = train_cnn_model(
    train_df, 
    test_df, 
    targets=TARGETS, 
    epochs=100,        
    batch_size=64
)

# 2. Get Predictions
df_1DCNN = run_cnn_inference(cnn_model, cnn_scaler, X_test_tensor, test_df['id'],targets=TARGETS)
print(df_1DCNN.head())

Loading data...
🔹 Preparing Data for Fixed 1D CNN...
✅ Vocab Size: 59
🔹 Training CNN for 100 epochs...
  Epoch 5: Avg Loss 0.6079
  Epoch 10: Avg Loss 0.4652
  Epoch 15: Avg Loss 0.3324
  Epoch 20: Avg Loss 0.2570
  Epoch 25: Avg Loss 0.1944
  Epoch 30: Avg Loss 0.1739
  Epoch 35: Avg Loss 0.1413
  Epoch 40: Avg Loss 0.1269
  Epoch 45: Avg Loss 0.1135
  Epoch 50: Avg Loss 0.1042
  Epoch 55: Avg Loss 0.0979
  Epoch 60: Avg Loss 0.0901
  Epoch 65: Avg Loss 0.0901
  Epoch 70: Avg Loss 0.0865
  Epoch 75: Avg Loss 0.0835
  Epoch 80: Avg Loss 0.0756
  Epoch 85: Avg Loss 0.0745
  Epoch 90: Avg Loss 0.0678
  Epoch 95: Avg Loss 0.0678
  Epoch 100: Avg Loss 0.0790
🔹 Running Inference...
Debug Prediction Sample (Scaled): [-0.19503985 -0.07964101  0.05771873  0.2871484   0.05512622]
          Tg        Tc   Density       FFV         Rg          id
0  90.965591  0.254168  0.987822  0.375195  16.490232  1109053969
1  90.209534  0.260009  1.008226  0.376963  17.255110  1422188626
2  90.144188  0.2548

# Mamba Architecture and Tokenizer 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
import numpy as np

# ==========================================
# 1. PURE PYTORCH MAMBA LAYER
# ==========================================
class PytorchMamba(nn.Module):
    """
    A pure PyTorch implementation of the Mamba layer (S6) compatible with mamba-ssm weights.
    """
    def __init__(self, d_model, d_state=16, d_conv=4, expand=2):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.expand = expand
        self.d_inner = int(self.expand * self.d_model)
        self.dt_rank = math.ceil(self.d_model / 16)

        # Input projection (x and z)
        self.in_proj = nn.Linear(d_model, self.d_inner * 2, bias=False)

        # 1D Convolution
        self.conv1d = nn.Conv1d(
            in_channels=self.d_inner,
            out_channels=self.d_inner,
            bias=True,
            kernel_size=d_conv,
            groups=self.d_inner,
            padding=d_conv - 1,
        )

        # SSM Projections (dt, B, C)
        self.x_proj = nn.Linear(self.d_inner, self.dt_rank + self.d_state * 2, bias=False)
        self.dt_proj = nn.Linear(self.dt_rank, self.d_inner, bias=True)

        # S6 Parameters
        # A_log is typically used in mamba-ssm to ensure A is negative
        self.A_log = nn.Parameter(torch.log(torch.arange(1, self.d_state + 1, dtype=torch.float32).repeat(self.d_inner, 1)))
        self.D = nn.Parameter(torch.ones(self.d_inner))

        # Output projection
        self.out_proj = nn.Linear(self.d_inner, d_model, bias=False)
        
        self.act = nn.SiLU()

    def selective_scan(self, u, delta, A, B, C, D):
        """
        Pure PyTorch implementation of the Selective Scan.
        u: (B, L, D_inner) - Input
        delta: (B, L, D_inner) - Time step
        A: (D_inner, D_state) - State matrix
        B: (B, L, D_state) - Input matrix
        C: (B, L, D_state) - Output matrix
        D: (D_inner) - Passthrough
        """
        batch_size, seq_len, d_inner = u.shape
        d_state = A.shape[1]
        
        # Discretize A: exp(delta * A)
        # delta: (B, L, D_inner), A: (D_inner, D_state) -> deltaA: (B, L, D_inner, D_state)
        deltaA = torch.exp(torch.einsum('bld,dn->bldn', delta, A))
        
        # Discretize B: delta * B
        # delta: (B, L, D_inner), B: (B, L, D_state) -> deltaB_u: (B, L, D_inner, D_state)
        # We also multiply by u here: (delta * B) * u
        deltaB_u = torch.einsum('bld,bln,bld->bldn', delta, B, u)
        
        # Recurrence Scan (Sequential loop is fast enough for short sequences ~180 tokens)
        x = torch.zeros((batch_size, d_inner, d_state), device=u.device)
        ys = []
        
        for t in range(seq_len):
            x = deltaA[:, t] * x + deltaB_u[:, t]
            y = torch.einsum('bdn,bn->bd', x, C[:, t])
            ys.append(y)
            
        y = torch.stack(ys, dim=1) # (B, L, D_inner)
        
        # Add residual D * u
        y = y + u * D
        return y

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        
        # 1. Projection: x -> (u, z)
        # shape: (B, L, 2*d_inner)
        xz = self.in_proj(x)
        x, z = xz.chunk(2, dim=-1)

        # 2. Convolution
        # Permute for Conv1d: (B, d_inner, L)
        x = x.transpose(1, 2)
        x = self.conv1d(x)[:, :, :seq_len] # Padding handling
        x = self.act(x)
        x = x.transpose(1, 2) # Back to (B, L, d_inner)

        # 3. SSM Projections
        # x_dbl: (B, L, dt_rank + 2*d_state)
        x_dbl = self.x_proj(x) 
        dt, B, C = torch.split(x_dbl, [self.dt_rank, self.d_state, self.d_state], dim=-1)
        
        # dt projection: (B, L, d_inner)
        dt = self.dt_proj(dt)
        dt = F.softplus(dt) # Ensure positive time step

        # 4. Selective Scan
        A = -torch.exp(self.A_log) # mamba-ssm uses -exp(A_log)
        y = self.selective_scan(x, dt, A, B, C, self.D)

        # 5. Gating and Output
        y = y * self.act(z)
        return self.out_proj(y)


# ==========================================
# 2. MODEL ARCHITECTURE
# ==========================================
class MegaMambaRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=512, d_state=16, d_conv=4, expand=2,
                 pad_idx=0, n_layers=3, dropout=0.25, head_type="mega"):
        super().__init__()
        self.pad_idx = pad_idx

        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)

        # Use our Custom PytorchMamba
        self.mamba_layers = nn.ModuleList([
            PytorchMamba(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand)
            for _ in range(n_layers)
        ])
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(d_model) for _ in range(n_layers)
        ])

        # Multi-head attention pooling
        self.attention_pool = nn.MultiheadAttention(d_model, num_heads=8, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        # Dynamic Head Architecture
        # Mega: d_model -> d_model -> d_model/2 -> 1
        # Regularized: d_model -> d_model/2 -> d_model/4 -> 1
        if head_type == "regularized":
            h1_dim = d_model // 2
            h2_dim = d_model // 4
        else: # "mega"
            h1_dim = d_model
            h2_dim = d_model // 2

        self.head1 = nn.Sequential(
            nn.Linear(d_model, h1_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        self.head2 = nn.Sequential(
            nn.Linear(h1_dim, h2_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        self.head3 = nn.Linear(h2_dim, 1)

    def forward(self, tokens, lengths=None):
        x = self.embed(tokens)
        for mamba, norm in zip(self.mamba_layers, self.layer_norms):
            residual = x
            x = mamba(x)
            x = norm(x)
            x = self.dropout(x)
            x = x + residual

        mask = (tokens != self.pad_idx)
        attn_mask = ~mask
        query = x.mean(dim=1, keepdim=True)
        pooled, _ = self.attention_pool(query, x, x, key_padding_mask=attn_mask)
        pooled = pooled.squeeze(1)

        h1 = self.head1(pooled)
        h2 = self.head2(h1)
        out = self.head3(h2).squeeze(-1)
        return out

# ==========================================
# 3. TOKENIZER LOGIC (Copied from notebook)
# ==========================================
def improved_smiles_tokenizer(smiles):
    tokens = []
    i = 0
    n = len(smiles)
    while i < n:
        char = smiles[i]
        if char == '[':
            end = smiles.find(']', i)
            if end != -1:
                tokens.append(smiles[i:end+1])
                i = end + 1
                continue
        if i < n - 1 and char.isupper() and smiles[i+1].islower():
            tokens.append(smiles[i:i+2])
            i += 2
            continue
        if char.isdigit():
            if i < n - 1 and smiles[i+1].isdigit():
                tokens.append(smiles[i:i+2])
                i += 2
            else:
                tokens.append(char)
                i += 1
            continue
        tokens.append(char)
        i += 1
    return tokens

In [15]:
import torch
print(f"Current PyTorch Version: {torch.__version__}")
print(f"CUDA Version: {torch.version.cuda}")

Current PyTorch Version: 2.6.0+cu124
CUDA Version: 12.4


# Training and Inference Loop

In [16]:
# ==========================================
# 4. SMART LOAD WEIGHTS & SETUP
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Your file paths
checkpoints_map = {
    "Tg": "/kaggle/input/mamba-model/pytorch/default/1/mamba_tg_regressor_mega.pt",
    "FFV": "/kaggle/input/mamba-model/pytorch/default/1/mamba_ffv_regressor_regularized.pt",
    "Tc": "/kaggle/input/mamba-model/pytorch/default/1/mamba_Tc_regressor_regularized.pt",
    "Density": "/kaggle/input/mamba-model/pytorch/default/1/mamba_Density_regressor_regularized.pt",
    "Rg": "/kaggle/input/mamba-model/pytorch/default/1/mamba_Rg_regressor_mega.pt" 
}

models = {} # Dictionary to store loaded models: models['Tg'], models['FFV'], etc.
configs = {} # Dictionary to store normalization stats: configs['Tg']['mean'], etc.

for target_name, path in checkpoints_map.items():
    print(f"\n--- Loading {target_name} model from: {path} ---")
    
    # 1. Load State Dict
    try:
        checkpoint = torch.load(path, map_location=device, weights_only=False)
    except:
        checkpoint = torch.load(path, map_location=device)
    
    state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
    
    # 2. Auto-detect Configuration
    # Detect d_model from embedding size
    d_model = state_dict['embed.weight'].shape[1]
    
    # Detect number of layers (count keys starting with mamba_layers.X)
    layer_indices = [int(k.split('.')[1]) for k in state_dict.keys() if k.startswith('mamba_layers.')]
    n_layers = max(layer_indices) + 1 if layer_indices else 0
    
    # Detect head type (Check output size of head1)
    head1_weight = state_dict['head1.0.weight']
    if head1_weight.shape[0] == d_model:
        head_type = "mega"
    else:
        head_type = "regularized"
        
    print(f"   Detected Config: d_model={d_model}, n_layers={n_layers}, head_type={head_type}")
    print(f"   Best Val R²: {checkpoint.get('val_r2', 'N/A')}")

    # 3. Initialize Model with detected config
    model = MegaMambaRegressor(
        vocab_size=len(checkpoint['itos']),
        d_model=d_model,
        d_state=16,
        d_conv=4,
        expand=2,
        pad_idx=checkpoint['stoi']['<pad>'],
        n_layers=n_layers,      # Dynamic
        head_type=head_type,    # Dynamic
        dropout=0.25
    ).to(device)
    
    model.load_state_dict(state_dict)
    model.eval()
    
    # Store model and stats
    models[target_name] = model
    configs[target_name] = {
        'stoi': checkpoint['stoi'],
        'pad_idx': checkpoint['stoi']['<pad>'],
        'mean': checkpoint['normalization_stats']['mean'],
        'std': checkpoint['normalization_stats']['std']
    }
    print(f"✅ {target_name} Model loaded successfully!")

def predict_property(smiles_list, target_name):
    """
    Predicts a specific property (Tg, FFV, etc.) for a list of SMILES
    """
    if target_name not in models:
        print(f"Model for {target_name} not found!")
        return None
        
    model = models[target_name]
    config = configs[target_name]
    stoi = config['stoi']
    pad_idx = config['pad_idx']
    
    # Encode
    encoded_batch = []
    for s in smiles_list:
        tokens = improved_smiles_tokenizer(str(s))
        enc = [stoi.get('<bos>', 1)] + [stoi.get(t, stoi.get('<pad>', 0)) for t in tokens] + [stoi.get('<eos>', 2)]
        encoded_batch.append(enc)
    
    # Pad
    max_len = max(len(x) for x in encoded_batch)
    padded = torch.full((len(encoded_batch), max_len), pad_idx, dtype=torch.long)
    for i, seq in enumerate(encoded_batch):
        padded[i, :len(seq)] = torch.tensor(seq)
        
    padded = padded.to(device)
    
    # Predict
    with torch.no_grad():
        preds_norm = model(padded)
        # Denormalize
        preds_orig = preds_norm * config['std'] + config['mean']
        
    return preds_orig.cpu().numpy()

# Usage Example:
# tg_preds = predict_property(test_df['SMILES'].tolist(), 'Tg')
# ffv_preds = predict_property(test_df['SMILES'].tolist(), 'FFV')


--- Loading Tg model from: /kaggle/input/mamba-model/pytorch/default/1/mamba_tg_regressor_mega.pt ---
   Detected Config: d_model=512, n_layers=3, head_type=mega
   Best Val R²: 0.6011854670997427
✅ Tg Model loaded successfully!

--- Loading FFV model from: /kaggle/input/mamba-model/pytorch/default/1/mamba_ffv_regressor_regularized.pt ---
   Detected Config: d_model=384, n_layers=2, head_type=regularized
   Best Val R²: 0.8560160577945413
✅ FFV Model loaded successfully!

--- Loading Tc model from: /kaggle/input/mamba-model/pytorch/default/1/mamba_Tc_regressor_regularized.pt ---
   Detected Config: d_model=384, n_layers=2, head_type=regularized
   Best Val R²: 0.7830454838336329
✅ Tc Model loaded successfully!

--- Loading Density model from: /kaggle/input/mamba-model/pytorch/default/1/mamba_Density_regressor_regularized.pt ---
   Detected Config: d_model=384, n_layers=2, head_type=regularized
   Best Val R²: 0.7320495098237798
✅ Density Model loaded successfully!

--- Loading Rg mode

In [17]:
import pandas as pd
import os

# 1. Load Test Data
test_path = "/kaggle/input/neurips-open-polymer-prediction-2025/test.csv"
test_df = pd.read_csv(test_path)

# Handle column name variations (usually 'smiles' or 'SMILES')
smiles_col = 'smiles' if 'smiles' in test_df.columns else 'SMILES'
print(f"Loaded Test Data: {len(test_df)} rows. Using column: '{smiles_col}'")

# 2. Define Targets
# These must match the keys used in your checkpoints_map
TARGETS = ["Tg", "FFV", "Tc", "Density", "Rg"]

# 3. Initialize df_mamba with IDs
df_mamba = test_df[['id']].copy()

print("🚀 Starting Mamba Inference...")

# 4. Predict each property
# This assumes you defined 'predict_property' and 'models' dictionary in the previous step
for target in TARGETS:
    print(f"  ... predicting {target}")
    
    # Get predictions using the helper function
    preds = predict_property(test_df[smiles_col].tolist(), target)
    
    # Store in dataframe
    if preds is not None:
        df_mamba[target] = preds
    else:
        print(f"  ⚠️ Warning: No model found for {target}, filling with 0")
        df_mamba[target] = 0.0

print("✅ df_mamba created successfully!")
print(df_mamba.head())

Loaded Test Data: 3 rows. Using column: 'SMILES'
🚀 Starting Mamba Inference...
  ... predicting Tg
  ... predicting FFV
  ... predicting Tc
  ... predicting Density
  ... predicting Rg
✅ df_mamba created successfully!
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  216.613007  0.375674  0.162934  1.165045  25.971161
1  1422188626  254.870255  0.376666  0.299271  1.058980  26.140137
2  2032016830   86.273544  0.351148  0.303936  1.105904  21.496885


# Average the Ensemble

In [18]:
import os
import pandas as pd

# ==========================================
# ROBUST ENSEMBLE (4 MODELS)
# ==========================================
print("🚀 Starting Ensemble Process...")

# 1. Re-Define Config & Targets to be safe
TARGETS = ["Tg", "FFV", "Tc", "Density", "Rg"]
AIMNET_SUB_PATH = '/kaggle/working/submission.csv' 
OUTPUT_PATH = '/kaggle/working/submission_ensemble.csv'

# 2. Check if AIMNet exists (Base Submission)
if not os.path.exists(AIMNET_SUB_PATH):
    raise FileNotFoundError(f"❌ AIMNet submission not found at {AIMNET_SUB_PATH}. Run Cell 15 first.")

# 3. Load All Dataframes
# We use 'id' as the index to ensure rows align correctly during addition
print("   Loading dataframes...")
df_aimnet = pd.read_csv(AIMNET_SUB_PATH).set_index('id')

# Check if other models exist in memory, if not, create dummies (zeros) to prevent crash
# (You should ensure Cell 17 and 19 ran before this!)
if 'df_xgb' not in locals():
    print("⚠️ df_xgb not found in memory! Did you run XGBoost? Skipping it.")
    df_xgb = pd.DataFrame(0, index=df_aimnet.index, columns=TARGETS)
    weight_xgb = 0
else:
    df_xgb = df_xgb.set_index('id')
    weight_xgb = 1

if 'df_1DCNN' not in locals():
    print("⚠️ df_1DCNN not found in memory! Did you run CNN? Skipping it.")
    df_1DCNN = pd.DataFrame(0, index=df_aimnet.index, columns=TARGETS)
    weight_cnn = 0
else:
    df_1DCNN = df_1DCNN.set_index('id')
    weight_cnn = 1

if 'df_mamba' not in locals():
    print("❌ df_mamba not found! Run the Mamba Prediction cell first.")
    raise NameError("df_mamba not defined")
else:
    df_mamba = df_mamba.set_index('id')
    weight_mamba = 1

# 4. Perform Ensemble (Average)
df_ensemble = df_aimnet.copy()
total_models = 1 + weight_xgb + weight_cnn + weight_mamba

print(f"   Ensembling {total_models} models...")

for target in TARGETS:
    # We use .get() or direct access. Since we aligned indices, simple + works.
    # Note: df_aimnet might have different column names if rename failed, checking:
    col_name = target if target in df_aimnet.columns else f"{target}_prediction"
    
    pred_sum = (
        df_aimnet[col_name] + 
        (df_xgb[target] if weight_xgb else 0) + 
        (df_1DCNN[target] if weight_cnn else 0) + 
        df_mamba[target]
    )
    
    df_ensemble[target] = pred_sum / float(total_models)

# 5. Save & Overwrite
df_ensemble = df_ensemble.reset_index() # Bring ID back as column
df_ensemble = df_ensemble[['id'] + TARGETS] # Ensure clean column order

df_ensemble.to_csv(OUTPUT_PATH, index=False)

import shutil
shutil.move(OUTPUT_PATH, '/kaggle/working/submission.csv')

print(f"✅ Ensemble Success! Combined {total_models} models.")
print(df_ensemble.head())

🚀 Starting Ensemble Process...
   Loading dataframes...
   Ensembling 4 models...
✅ Ensemble Success! Combined 4 models.
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  127.135743  0.371458  0.215679  1.180964  21.451319
1  1422188626  149.477227  0.377099  0.260330  1.127054  20.430720
2  2032016830   77.992380  0.353393  0.280315  1.112848  18.878126
